In [ ]:
# pip install scikit-plot

Importing dependencies

In [ ]:
import numpy as np
import scikitplot as skplt
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, roc_curve

In [ ]:
import tensorflow as tf
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import Flatten
from keras.layers import MaxPool2D
from keras.models import Sequential
from tensorflow.keras.applications import VGG16

Importing dataset - [DATASET LINK](https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz)

In [ ]:
(train_x,train_y),(test_x,test_y) = tf.keras.datasets.cifar10.load_data()

In [ ]:
data_x = np.append(train_x,test_x,axis=0)
data_y = np.append(train_y,test_y,axis=0)

Training for 80%, Validation 10%, and test for 10%.

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(data_x,data_y,train_size=0.8)
validate_x, test_x, validate_y, test_y = train_test_split(test_x,test_y,test_size=0.5)

In [ ]:
def Accuracy(test_y,predict_y,proba_y):
  accuracy = accuracy_score(test_y, predict_y)
  print('Accuracy for Testing: %.3f' % accuracy)

  print(classification_report(test_y,predict_y,digits=3))

  print('ROC Curve')
  skplt.metrics.plot_roc(test_y, proba_y)
  plt.show()

KNN

In [ ]:
KNN = KNeighborsClassifier(n_neighbors=13)

KNN.fit(np.reshape(train_x,(train_x.shape[0],-1)),train_y.ravel())

proba_y = KNN.predict_proba(np.reshape(test_x,(test_x.shape[0],-1)))
predict_y = np.argmax(proba_y,axis=1)

Accuracy(test_y.ravel(),predict_y,proba_y)

SVM

In [ ]:
classifier = SVC(kernel='poly',decision_function_shape='ovr',probability=True)

scaler = StandardScaler()

classifier.fit(scaler.fit_transform(np.reshape(train_x,(train_x.shape[0],-1))),train_y.ravel())

proba_y = classifier.predict_proba(scaler.transform(np.reshape(test_x,(test_x.shape[0],-1))))
predict_y = np.argmax(proba_y,axis=1)

Accuracy(test_y.ravel(),predict_y,proba_y)

CNN

In [ ]:
base = VGG16(include_top=False,input_shape=(32,32,3),classes=10)

model = Sequential()
model.add(base)
model.add(Flatten())
model.add(Dense(1024,activation=('relu'),input_dim=512))
model.add(Dense(512,activation=('relu'))) 
model.add(Dense(256,activation=('relu'))) 
model.add(Dense(128,activation=('relu')))
model.add(Dense(10,activation=('softmax')))

model.summary()

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='RMSprop',metrics=['accuracy'])

model.fit(train_x,train_y.ravel(), epochs=5, batch_size=16, validation_data=(validate_x, validate_y.ravel()), verbose=1)

In [ ]:
proba_y = model.predict(test_x)
predict_y = np.argmax(proba_y,axis=1)

Accuracy(test_y.ravel(),predict_y,proba_y)